In [198]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

% matplotlib inline

In [199]:
length = 9

# Base peptide sequences

In [200]:
if   length == 8:
    base = 'WTHPQFAT'
elif length == 9:
    base = 'LQWHPQAGK'
elif length == 12:
    base = 'GKFPIPLGKQSG'
elif length == 13:
    base = 'NGQFQVWIPGAQK'
else:
    print 'No base of such length'
    
aminoacids = ['A', 'C', 'D', 'E', 'F',
              'G', 'H', 'I', 'K', 'L',
              'M', 'N', 'P', 'Q', 'R',
              'S', 'T', 'V', 'W', 'Y']

# Creating list of peptide names

In [201]:
def namelist(base, n):
    names = []
    for a in aminoacids:
        names.append(base[:n - 1] + a + base[n:])
    return names

# Logarithmize errorbars

In [202]:
def logarithmize(x, xbar):
    x = np.array(x)
    xbar = np.array(xbar)

    x_log = np.log10(x)
    xbar_upper = np.log10(x + xbar) - x_log
    xbar_lower = x_log - np.log10(x - xbar)
    
    return x_log, xbar_lower, xbar_upper

# Tweak errorbars of bound values

In [203]:
def bound_bar(bar, flag):
    for i, e in enumerate(flag):
        if e:
            bar[i] = 10
    return bar

# Subsetting data

In [204]:
def reorder(data, names):
    return data

In [205]:
def subset(base, n):
    names = namelist(base, n)
    data = pd.read_csv('04 Aggregated dR RT Streptavidin length 8 and 9.csv', header = 0)
    data = data[data['Peptide Length'] == len(base)]
    data = data[data['Peptide'].isin(names)]
    assert len(data) == 20
    data = reorder(data, names)
    return data

In [206]:
def plot_series(data, filename):


    x       = data['Ka Filtered'].tolist()
    xbar    = data['Ka Filtered Err'].tolist()

    ylabels = data['Peptide'].tolist()
    y       = range(len(ylabels), 0, -1)

    xlolims = data['Upper Bound Kd'].tolist()
    xuplims = data['Lower Bound Kd'].tolist()

    width = 6
    height = 0.36 * (len(y) + 1.4) + 1.1

    xlim_left = -5
    xlim_right = 2

    # rescale to log10
    x, xbar_lower, xbar_upper = logarithmize(x, xbar)

    # tweak points with negative lower errorbars
    for i, e in enumerate(xbar_lower):
        if np.isnan(e):
            xuplims[i] = True
            x[i] = x[i] + xbar_upper[i]
            xbar_upper[i] = 0

    # tweak errorbars of upper and lower bounds
    for i, e in enumerate(xuplims):
        if e:
            xbar_lower[i] = x[i] - xlim_left

    for i, e in enumerate(xlolims):
        if e:
            xbar_upper[i] = xlim_right - x[i]   

    # set color
    color = ['k'] * len(x)
    for i,e in enumerate(xuplims):
        if e or xlolims[i]:
            color[i] = 'r'

    # set marker
    marker = ['o'] * len(x)
    for i in range(len(x)):
        if   xlolims[i]:
            marker[i] = '>'
        elif xuplims[i]:
            marker[i] = '<'

    # set markersize
    markersize = [6] * len(x)
    for i,e in enumerate(xuplims):
        if e or xlolims[i]:
            markersize[i] = 9

    fig = plt.figure(figsize = (width, height))
    ax = fig.add_subplot(1, 1, 1)

    for i in range(len(x)):
        # plot points
        ax.errorbar(x[i], y[i],
                    xerr = [[xbar_lower[i]], [xbar_upper[i]]],
                    xlolims = xlolims[i],
                    xuplims = xuplims[i],

                    marker = marker[i],
                    color = color[i],
                    markersize = markersize[i],
                    ls = 'none',
                    markeredgecolor = 'none')

        # x axis labels
        for label in ax.get_xticklabels(): 
            label.set_fontname('DejaVu Sans Mono')
            label.set_fontsize(14)

        # y axis labels
        for label in ax.get_yticklabels():
            label.set_fontname('DejaVu Sans Mono') # alternatively: 'Droid Sans Mono', 'FreeMono' or 'Liberation Mono'
            label.set_fontsize(14)
            if label.get_text() == base:
                label.set_weight('black')
            else:
                label.set_color('0.2')

        # axis limits
        ax.set_xlim(left = xlim_left, right = xlim_right)
        ax.set_ylim(bottom = min(y) - 0.7, top = max(y) + 0.7)

        # axis ticks
        for tick in ax.get_xaxis().get_major_ticks():
            tick.set_pad(6)
        for tick in ax.get_yaxis().get_major_ticks():
            tick.set_pad(8)

    plt.yticks(y, ylabels)
    plt.grid(alpha = 0.5)

    plt.subplots_adjust(left = 1.8 / width,
                        right = 1 - 0.3 / width,
                        bottom = 0.8 / height,
                        top = 1 - 0.3 / height)

    plt.xlabel('$\log_{10}{\ K_a}$', labelpad = 12, fontsize = 18)
    plt.axvspan(-5, -4.4, alpha = 0.1, facecolor = 'k', edgecolor = 'none')     

#    plt.show()
    plt.savefig(filename)

# Important note about handling error bars
In total, I have four types of data points on my plots:

- Normal points. These are reactive peptides, for which $K_d^{(best)} = K_d$. Fitting parameter $K_a = \dfrac{1}{K_d^{(best)}}$ is a true value of association constant and the errorbar is smaller than the value itself. Such points are plotted in black.


- Normal points with large errorbars. Just like in previous category, $K_d^{(best)} = K_d$, but this time fitting error of the $K_d$ (and $K_a$) is larger than fitted value of $K_d$ (or $K_a$) itself. Errorbar would go all the way to zero on normal plot and all the way to $-\inf$ on logarithmic plot. I chose to plot such points as **upper boundaries** of $K_a$ with red triangle at $K_a + \text{Err }(K_a)$ and line stretching left.


- Non-reactive peptides. These points are plotted as red triangles at the detection limit of $K_a = 30000 \text{ nM}$ with a line stretching left. Detection limit of association constant provides an **upper boundary** for $K_a$.


- Points with $k_{off}$ below detection limit. For these points $K_d^{(best)} > K_d$. Fitted value of $K_d$ is below detection limit of dissociation rate and does not represent true dissociation constant. We should use $K_a = \dfrac{1}{K_d^{(best)}}$ as **lower limit** of association constant. Since $\text{Err }(K_d)$ is an error of fitted dissociation constant, which is not accurate, I ignore this errorbar altogether.

In [211]:
for i in range(1, length + 1):
    filename = 'base ' + str(length) + ', letter ' + str(i) + '.png'
    plot_series(subset(base, i), filename)
    
plt.clf()
plt.close('all')

In [207]:
plt.clf()
plt.close('all')

In [209]:
# print subset(base, 6)

In [212]:
# plot_series(subset(base, 6), 'a')